In [1]:
import pyodbc
import pandas as pd

In [2]:
def connect():

    # Define the connection string
# Define the connection string
    server = 'tcp:sqlsrv-stage.database.windows.net,1433'
    database = 'sqldb_stage'
    username = 'hababi@sqlsrv-stage'
    password = 'Hassan&123'
    driver = '{ODBC Driver 18 for SQL Server}'  # This driver may vary depending on your setup


    # Create the connection
    try:
        conn = pyodbc.connect(f'Driver={driver};Server={server};Database={database};Uid={username};Pwd={password};Encrupt=yes;TrustServerCertificate=no;Connection Timeout=30;')        
        print('Connected.')
        
        return conn

    except Exception as e:
        print(f'Error connecting to SQL Server: {str(e)}')


In [3]:
def create_jobdb():
    conn = connect()
    cursor = conn.cursor()

    # SQL command to create a table
    create_table_sql = '''
    CREATE TABLE jobs_data (
        id INT IDENTITY(1,1) PRIMARY KEY,
        job_link NVARCHAR(MAX),
        job_name NVARCHAR(255),
        job_text NVARCHAR(MAX),
        job_company NVARCHAR(255),
        job_location NVARCHAR(255),
        job_type NVARCHAR(50),
        job_date DATE,
        skills NVARCHAR(MAX)
    );
    '''

    # Executing the SQL command
    cursor.execute(create_table_sql)
    conn.commit()

    # Closing the connection
    cursor.close()
    conn.close()

    print("Table created successfully.")


In [4]:
create_jobdb()

Connected.


ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]There is already an object named 'jobs_data' in the database. (2714) (SQLExecDirectW)")

In [5]:
def insert_data(path):
    df = pd.read_csv(path)
    df = df.dropna()
    
    
    # query
    insert_query = """
        INSERT INTO jobs_data (job_link, job_name, job_text, job_company, job_location, job_type, job_date, skills)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """
    

    conn = None

    i = 0
    for _, row in df.iterrows():
        if not conn:
            conn = connect()
            cursor = conn.cursor()
        try:
            cursor.execute(insert_query, 
                        row['job_link'], 
                        row['job_name'], 
                        row['job_text'], 
                        row['job_company'], 
                        row['job_location'], 
                        row['job_type'], 
                        row['job_date'],
                        row['skills']
                        )
            i+=1
            print(i,"rows inserted",end="\r")
        except pyodbc.Error as pe:
            print("Error:", pe)
            if pe.args[0] == "08S01":  
                try:
                    conn.close()
                except:
                    pass
                conn = None
                continue
    
        
    
    # Commit the transaction
    conn.commit()
    
    # Close the connection
    cursor.close()
    conn.close()


In [6]:
insert_data("jobs_data.csv")

Connected.


SystemError: <class 'pyodbc.Error'> returned a result with an exception set

In [11]:
import psycopg2
import pandas as pd

def connect():
    try:
        conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=aminecss")
        print('DB connected successfully')
        return conn
    except Exception as e:
        print(f"Error connecting to DB: {e}")
        raise

In [12]:
conn=connect()
cursor = conn.cursor()

DB connected successfully
